In [1]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from model2vec import StaticModel
from langchain.embeddings.base import Embeddings
import hashlib

In [2]:
class Model2VecEmbeddings(Embeddings):
        """Wrapper para o Model2Vec como Embeddings do LangChain"""
        def __init__(self, model_name: str):
            self.model = StaticModel.from_pretrained(model_name)

        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return self.model.encode(texts).tolist()
        
        def embed_query(self, text: str) -> List[float]:
            return self.model.encode([text]).tolist()[0]
        
class DeduplicationProcessor:
    def __init__(self, model_name: str = "minishlab/potion-base-2M", similarity_threshold: int = 0.85):
        """Inicializa o processador com o modelo de embeddings"""
        self.embedder = Model2VecEmbeddings(model_name)
        self.similarity_threshold = similarity_threshold  # Ajuste conforme necessário
        self.min_length = 10  # Comentários muito curtos serão ignorados    

    def load_data(self, parquet_path: str) -> pd.DataFrame:
        """Carrega os dados do arquivo parquet"""
        df = pd.read_parquet(parquet_path)
        
        # Pré-filtro básico
        df = df.dropna(subset=['comment_cleaned'])
        df = df[df['comment_cleaned'].str.len() >= self.min_length]
        
        return df.reset_index(drop=True)

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Gera embeddings para todos os textos"""
        print("Gerando embeddings...")
        return np.array(self.embedder.embed_documents(texts))

    def find_similar_pairs(self, embeddings: np.ndarray) -> List[Tuple[int, int]]:
        """Identifica pares similares usando similaridade de cosseno"""
        print("Calculando similaridades...")
        sim_matrix = cosine_similarity(embeddings)
        np.fill_diagonal(sim_matrix, 0)  # Ignora auto-similaridade
        
        similar_pairs = []
        n = sim_matrix.shape[0]
        
        # Encontra pares acima do threshold
        for i in tqdm(range(n), desc="Processando similaridades"):
            for j in range(i+1, n):
                if sim_matrix[i, j] > self.similarity_threshold:
                    similar_pairs.append((i, j))
        
        return similar_pairs

    def cluster_similar_comments(self, df: pd.DataFrame, similar_pairs: List[Tuple[int, int]]) -> pd.DataFrame:
        """Agrupa comentários similares e mantém apenas um representante"""
        print("Agrupando comentários similares...")
        clusters = []
        visited = set()
        
        # Cria clusters de similaridade
        for i, j in similar_pairs:
            if i not in visited and j not in visited:
                clusters.append({i, j})
                visited.update({i, j})
            elif i in visited and j not in visited:
                for cluster in clusters:
                    if i in cluster:
                        cluster.add(j)
                        visited.add(j)
                        break
            elif j in visited and i not in visited:
                for cluster in clusters:
                    if j in cluster:
                        cluster.add(i)
                        visited.add(i)
                        break
        
        # Seleciona representantes (o comentário mais longo de cada cluster)
        to_remove = set()
        for cluster in clusters:
            cluster_texts = [(idx, df.loc[idx, 'comment_cleaned']) for idx in cluster]
            # Ordena por comprimento e seleciona o mais longo
            cluster_texts.sort(key=lambda x: len(x[1]), reverse=True)
            representative = cluster_texts[0][0]
            to_remove.update([idx for idx, _ in cluster_texts[1:]])
        
        # Cria coluna de hash para identificação de duplicatas exatas
        df['text_hash'] = df['comment_cleaned'].apply(
            lambda x: hashlib.md5(x.strip().lower().encode()).hexdigest()
        )
        
        # Remove duplicatas exatas primeiro
        df = df.drop_duplicates(subset=['text_hash'], keep='first')
        
        # Remove comentários similares não representativos
        df = df.drop(index=list(to_remove)).reset_index(drop=True)
        
        return df.drop(columns=['text_hash'])

    def process(self, input_path: str, output_path: str) -> pd.DataFrame:
        """Pipeline completo de processamento"""
        # 1. Carregar dados
        df = self.load_data(input_path)
        print(f"Total de comentários inicial: {len(df)}")
        
        # 2. Gerar embeddings
        embeddings = self.generate_embeddings(df['comment_cleaned'].tolist())
        
        # 3. Encontrar pares similares
        similar_pairs = self.find_similar_pairs(embeddings)
        print(f"Pares similares encontrados: {len(similar_pairs)}")
        
        # 4. Clusterizar e remover duplicatas
        df_dedup = self.cluster_similar_comments(df, similar_pairs)
        print(f"Total de comentários após deduplicação: {len(df_dedup)}")
        print(f"Comentários removidos: {len(df) - len(df_dedup)}")
        
        # 5. Salvar resultados
        df_dedup.to_parquet(output_path, index=False)
        print(f"Dados processados salvos em {output_path}")
        
        return (df_dedup, similar_pairs, df, embeddings)

INPUTS

In [8]:
if __name__ == "__main__":
    processor = DeduplicationProcessor(similarity_threshold=0.7)
    
    # Arquivos de entrada e saída
    input_parquet = "../tests\dataset_train_with_sentiment_positives_to_trim.parquet"
    output_parquet = "../tests\dataset_train_with_sentiment_positives_to_trim.parquet"
    
    # Executar pipeline
    (df_dedup, similar_pairs, df, embeddings) = processor.process(input_parquet, output_parquet)

Total de comentários inicial: 341
Gerando embeddings...
Calculando similaridades...


Processando similaridades: 100%|██████████| 341/341 [00:00<00:00, 24997.95it/s]

Pares similares encontrados: 45
Agrupando comentários similares...
Total de comentários após deduplicação: 298
Comentários removidos: 43
Dados processados salvos em ../tests\dataset_train_with_sentiment_positives_to_trim.parquet


In [4]:
def get_similar_comments(df: pd.DataFrame, 
                        similar_pairs: List[Tuple[int, int]], 
                        embeddings: np.ndarray,
                        top_n: int = 20000) -> List[Dict]:
    """
    Retorna uma lista dos comentários similares com seus textos e scores de similaridade.
    
    Args:
        df: DataFrame com os comentários (deve ter coluna 'comment_cleaned')
        similar_pairs: Lista de tuplas com índices dos pares similares
        embeddings: Matriz de embeddings dos comentários
        top_n: Quantos pares similares retornar (ordenados por similaridade)
    
    Returns:
        Lista de dicionários com pares similares e informações
    """
    # Calcula similaridades para todos os pares
    similar_comments = []
    for i, j in similar_pairs:
        sim_score = cosine_similarity(
            embeddings[i].reshape(1, -1), 
            embeddings[j].reshape(1, -1)
        )[0][0]
        similar_comments.append({
            'index1': i,
            'index2': j,
            'text1': df.loc[i, 'comment_cleaned'],
            'text2': df.loc[j, 'comment_cleaned'],
            'similarity': sim_score,
            'length_diff': abs(len(df.loc[i, 'comment_cleaned']) - len(df.loc[j, 'comment_cleaned']))
        })
    
    # Ordena por similaridade decrescente
    similar_comments.sort(key=lambda x: x['similarity'], reverse=True)
    
    # Retorna apenas os top_n mais similares
    return similar_comments[:top_n]

# Função para visualizar os resultados de forma elegante
def print_similar_pairs(similar_comments: List[Dict]):
    """Imprime os pares similares de forma formatada"""
    print(f"\n{'='*50}")
    print(f"{'PAIR':<6} | {'SIMILARITY':<10} | {'TEXT 1':<50} | {'TEXT 2':<50}")
    print(f"{'-'*120}")
    for i, pair in enumerate(similar_comments, 1):
        text1_short = (pair['text1'][:47] + '...') if len(pair['text1']) > 50 else pair['text1']
        text2_short = (pair['text2'][:47] + '...') if len(pair['text2']) > 50 else pair['text2']
        print(f"{i:<6} | {pair['similarity']:.4f}    | {text1_short:<50} | {text2_short:<50}")
    print(f"{'='*120}\n")

In [5]:
comments = get_similar_comments(df=df, similar_pairs=similar_pairs, embeddings=embeddings)

In [6]:
print_similar_pairs(comments)


PAIR   | SIMILARITY | TEXT 1                                             | TEXT 2                                            
------------------------------------------------------------------------------------------------------------------------
1      | 0.8942    | the atmosphere is great ! ! !                      | the design and atmosphere is just as good .       
2      | 0.8902    | the pizza was really good .                        | the pizza was pretty good and huge .              
3      | 0.8628    | the pizza was really good .                        | great pizza and fantastic service .               
4      | 0.8572    | ballato 's is consistently delicious authentic ... | the food is authentic italian - delicious !       

